<span style="color:blue"> Бизнес-задача: 
- Предложить стратегию инвестиций в разработку игр, выявив связь между характеристиками игр (жанр, средний рейтинг, платформа) и их коммерческим успехом (продажи, выручка). </span>

In [26]:
import pandas as pd

In [43]:
games_rate_data = pd.read_csv('games.csv')
games_revenue_data = pd.read_csv('gamalytic_games.csv')

games_rate_data['esrb_rating'] = games_rate_data['esrb_rating'].fillna('No age rating')
games_rate_data = games_rate_data.rename(columns={'esrb_rating': 'age_rating'})

games_revenue = games_revenue_data.drop(columns=['steamId', 'unreleased', 'earlyAccess', 'publisherClass', 'publishers', 'headerUrl', 'steamUrl'])

games_rate_data

,name,released,rating,age_rating,genres,platforms
0,Grand Theft Auto V,2013-09-17,4.47,Mature,Action,"PC, PlayStation 5, Xbox Series S/X, PlayStatio..."
1,The Witcher 3: Wild Hunt,2015-05-18,4.64,Mature,"Action, RPG","PlayStation 5, Xbox Series S/X, macOS, PlaySta..."
2,Portal 2,2011-04-18,4.58,Everyone 10+,"Shooter, Puzzle","PlayStation 3, PC, Xbox 360, Linux, macOS, Xbo..."
3,Counter-Strike: Global Offensive,2012-08-21,3.57,Mature,Shooter,"PC, Linux, Xbox 360, PlayStation 3"
4,Tomb Raider (2013),2013-03-05,4.06,Mature,Action,"PlayStation 3, Xbox 360, macOS, PC, Xbox One, ..."
...,...,...,...,...,...,...
13995,Undefeated,2015-04-03,0.00,No age rating,"Action, Adventure, RPG, Strategy, Casual, Indie","PC, Android"
13996,NightmareZ,2016-09-08,0.00,No age rating,"Action, Indie",PC
13997,FERIT Simulator,2017-02-23,0.00,No age rating,Educational,PC
13998,BAD END,2013-12-13,2.17,Teen,"Adventure, Indie","iOS, PC"
